BT4014 MMA Project

In [1]:
import pandas as pd
import datetime

In [2]:
# Using the basic stats as outlined in the dataset description (except urls)
masterMLpublic = pd.read_csv("/kaggle/input/mma-differentials-and-elo/masterMLpublic.csv",
                            usecols = [
                                "date", "result", "fighter", "opponent", "division",
                                "stance", "dob", "method", "total_comp_time", "round", 
                                "time", "referee", "time_format", "reach", "height",
                                "age", "knockdowns", "sub_attempts", "reversals", "control",
                                "takedowns_landed", "takedowns_attempts",
                                "sig_strikes_landed", "sig_strikes_attempts",
                                "total_strikes_landed", "total_strikes_attempts",
                                "head_strikes_landed", "head_strikes_attempts",
                                "body_strikes_landed", "body_strikes_attempts",
                                "leg_strikes_landed", "leg_strikes_attempts",
                                "distance_strikes_landed", "distance_strikes_attempts",
                                "clinch_strikes_landed", "clinch_strikes_attempts",
                                "ground_strikes_landed", "ground_strikes_attempts",
                                "KO_losses", "days_since_last_comp", "lose_streak",
                                "win_streak", "win_loss_ratio", "total_comp_time", "stamina",
                                "num_fights", "trueskill", "elo"
                            ])
# Convert the date columns from object into datetime
masterMLpublic["date"] = pd.to_datetime(masterMLpublic["date"]).dt.date
masterMLpublic["dob"] = pd.to_datetime(masterMLpublic["dob"]).dt.date
# removing draws because nobody wins
masterMLpublic = masterMLpublic[masterMLpublic["method"] != "DRAW"]
# removing dqs because they are not standard wins
masterMLpublic = masterMLpublic[masterMLpublic["method"] != "DQ"]
masterMLpublic

,date,result,fighter,opponent,stance,dob,division,method,round,time,...,ground_strikes_attempts,num_fights,win_streak,lose_streak,win_loss_ratio,KO_losses,total_comp_time,stamina,elo,trueskill
0,1994-03-11,0,sean daugherty,scott morris,NaN,1975-12-04,Open Weight,SUB,1,0:20,...,0,1.0,0,1,0.000000,0,20.0,0.000000,1147.756618,20.794729
1,1994-03-11,1,scott morris,sean daugherty,Orthodox,NaT,Open Weight,SUB,1,0:20,...,0,1.0,1,0,1.000000,0,20.0,0.000000,1252.243382,29.205271
2,1994-03-11,0,ray wizard,patrick smith,NaN,NaT,Open Weight,SUB,1,0:58,...,0,1.0,0,1,0.000000,0,58.0,0.000000,1147.164966,20.794729
3,1994-03-11,1,patrick smith,ray wizard,Orthodox,1963-08-28,Open Weight,SUB,1,0:58,...,0,1.0,1,0,1.000000,0,58.0,0.000000,1249.442138,29.205271
4,1994-03-11,0,david levicki,johnny rhodes,NaN,NaT,Open Weight,KO/TKO,1,12:13,...,1,1.0,0,1,0.000000,1,733.0,0.000000,1141.838344,20.794729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14125,2023-04-15,1,azamat murzakanov,dustin jacoby,Southpaw,1989-04-12,Light Heavyweight,U-DEC,3,5:00,...,0,3.0,3,0,1.000000,0,2222.0,1.000000,1341.922237,35.760448
14126,2023-04-15,0,billy quarantillo,edson barboza,Orthodox,1988-12-08,Featherweight,KO/TKO,1,2:37,...,0,8.0,0,1,0.625000,1,5352.0,0.600000,1264.744398,31.425332
14127,2023-04-15,1,edson barboza,billy quarantillo,Orthodox,1986-01-21,Featherweight,KO/TKO,1,2:37,...,1,28.0,1,0,0.607143,4,18947.0,0.631579,1350.043973,34.795952
14128,2023-04-15,0,arnold allen,max holloway,Southpaw,1994-01-22,Featherweight,U-DEC,5,5:00,...,0,11.0,0,1,0.909091,0,8823.0,0.888889,1489.695647,39.967508


In [3]:
masterMLpublic['division'].value_counts()

division
Lightweight              2422
Welterweight             2366
Middleweight             1868
Featherweight            1306
Heavyweight              1286
Light Heavyweight        1244
Bantamweight             1162
Flyweight                 574
Women's Strawweight       510
Women's Flyweight         376
Women's Bantamweight      358
Open Weight               200
Catch Weight              112
Women's Featherweight      52
Super Heavyweight           2
Name: count, dtype: int64

In [4]:
lightweight = masterMLpublic[masterMLpublic["division"] == "Lightweight"]
lightweight

,date,result,fighter,opponent,stance,dob,division,method,round,time,...,ground_strikes_attempts,num_fights,win_streak,lose_streak,win_loss_ratio,KO_losses,total_comp_time,stamina,elo,trueskill
198,1997-02-07,0,rainy martinez,jerry bohlander,Orthodox,NaT,Lightweight,SUB,1,1:24,...,0,1.0,0,1,0.000000,0,84.0,0.000000,1155.936792,21.246039
199,1997-02-07,1,jerry bohlander,rainy martinez,Orthodox,1974-02-12,Lightweight,SUB,1,1:24,...,0,4.0,2,0,0.750000,1,1858.0,0.000000,1292.341803,29.965180
200,1997-02-07,0,wallid ismail,yoshiki takahashi,Orthodox,1968-02-23,Lightweight,U-DEC,2,3:00,...,0,1.0,0,1,0.000000,0,900.0,0.000000,1147.756618,20.794729
201,1997-02-07,1,yoshiki takahashi,wallid ismail,Southpaw,1969-03-13,Lightweight,U-DEC,2,3:00,...,10,1.0,1,0,1.000000,0,900.0,0.000000,1252.243382,29.205271
206,1997-02-07,0,nick sanzo,jerry bohlander,NaN,NaT,Lightweight,SUB,1,0:39,...,0,1.0,0,1,0.000000,0,39.0,0.000000,1162.881928,21.710767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14055,2023-03-18,1,justin gaethje,rafael fiziev,Orthodox,1988-11-14,Lightweight,M-DEC,3,5:00,...,0,11.0,1,0,0.636364,2,6670.0,0.600000,1491.524082,38.809578
14108,2023-04-15,0,lando vannata,daniel zellhuber,Orthodox,1992-03-14,Lightweight,U-DEC,3,5:00,...,2,13.0,0,2,0.307692,0,9089.0,0.222222,1148.578353,26.265970
14109,2023-04-15,1,daniel zellhuber,lando vannata,Switch,1999-07-07,Lightweight,U-DEC,3,5:00,...,51,2.0,1,0,0.500000,0,1800.0,0.500000,1200.212327,26.476804
14118,2023-04-15,0,clay guida,rafa garcia,Orthodox,1981-12-08,Lightweight,U-DEC,3,5:00,...,0,35.0,0,1,0.514286,2,24276.0,0.521739,1178.957220,31.316986


In [5]:
welterweight = masterMLpublic[masterMLpublic["division"] == "Welterweight"]
welterweight

,date,result,fighter,opponent,stance,dob,division,method,round,time,...,ground_strikes_attempts,num_fights,win_streak,lose_streak,win_loss_ratio,KO_losses,total_comp_time,stamina,elo,trueskill
432,2000-06-09,0,adrian serrano,shonie carter,NaN,1963-12-23,Welterweight,U-DEC,2,5:00,...,0,1.0,0,1,0.000000,0,600.0,0.000000,1150.172166,21.682265
433,2000-06-09,1,shonie carter,adrian serrano,Southpaw,1972-05-03,Welterweight,U-DEC,2,5:00,...,6,2.0,2,0,1.000000,0,1200.0,0.000000,1281.078725,31.678480
438,2000-06-09,0,marcelo aguiar,matt hughes,Orthodox,NaT,Welterweight,KO/TKO,1,4:34,...,2,1.0,0,1,0.000000,1,274.0,0.000000,1145.818567,21.514298
439,2000-06-09,1,matt hughes,marcelo aguiar,Switch,1973-10-13,Welterweight,KO/TKO,1,4:34,...,29,2.0,2,0,1.000000,0,1174.0,1.000000,1297.903893,31.133798
442,2000-06-09,0,john alessio,pat miletich,Orthodox,1979-07-05,Welterweight,SUB,2,1:43,...,0,1.0,0,1,0.000000,0,403.0,0.000000,1164.143224,23.188517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14063,2023-03-25,1,trevin giles,preston parsons,Orthodox,1992-08-06,Welterweight,S-DEC,3,5:00,...,16,11.0,2,0,0.636364,2,7704.0,0.750000,1233.022916,30.432315
14094,2023-04-08,0,santiago ponzinibbio,kevin holland,Orthodox,1986-09-26,Welterweight,KO/TKO,3,3:16,...,0,17.0,0,1,0.647059,3,11235.0,0.636364,1356.768090,34.596216
14095,2023-04-08,1,kevin holland,santiago ponzinibbio,Orthodox,1992-11-05,Welterweight,KO/TKO,3,3:16,...,4,18.0,1,0,0.611111,1,11976.0,0.600000,1399.070371,34.450600
14098,2023-04-08,0,jorge masvidal,gilbert burns,Orthodox,1984-11-12,Welterweight,U-DEC,3,5:00,...,0,22.0,0,4,0.545455,1,17618.0,0.400000,1418.707657,35.283501


In [6]:
# Assuming masterMLpublic is a pandas DataFrame
na_counts = welterweight.isna().sum()
na_counts

date                           0
result                         0
fighter                        0
opponent                       0
stance                        10
dob                           13
division                       0
method                         0
round                          0
time                           0
referee                        2
time_format                    0
reach                        205
age                           13
height                         0
knockdowns                     0
days_since_last_comp           0
sub_attempts                   0
reversals                      0
control                        0
takedowns_landed               0
takedowns_attempts             0
sig_strikes_landed             0
sig_strikes_attempts           0
total_strikes_landed           0
total_strikes_attempts         0
head_strikes_landed            0
head_strikes_attempts          0
body_strikes_landed            0
body_strikes_attempts          0
leg_strike

Data Cleaning

In [7]:
#null values
average_reach_welterweight = welterweight[welterweight['division'] == 'Welterweight']['reach'].mean()
welterweight['reach'].fillna(average_reach_welterweight, inplace=True)

average_age_welterweight = welterweight[welterweight['division'] == 'Welterweight']['age'].mean()
welterweight['age'].fillna(average_age_welterweight, inplace=True)

average_height_welterweight = welterweight[welterweight['division'] == 'Welterweight']['height'].mean()
welterweight['height'].fillna(average_height_welterweight, inplace=True)

#replace
def time_to_seconds(time_str):
    if pd.isnull(time_str):
        return np.nan
    minutes, seconds = map(int, time_str.split(':'))
    return minutes * 60 + seconds

specific_value = 216.1080438291005
welterweight.loc[welterweight['days_since_last_comp'] == specific_value, 'days_since_last_comp'] = welterweight['age']


welterweight['time_seconds'] = welterweight['time'].apply(time_to_seconds)

#remove columns
columns_to_ignore = ['date', 'dob', 'division', 'referee', 'time', 'time_format']
welterweight.drop(columns=columns_to_ignore, inplace=True)


/tmp/ipykernel_19/3807069755.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  welterweight['reach'].fillna(average_reach_welterweight, inplace=True)
/tmp/ipykernel_19/3807069755.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  welterweight['age'].fillna(average_age_welterweight, inplace=True)
/tmp/ipykernel_19/3807069755.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  welterweight['height'].fillna(average_height_welterweight,

In [8]:
cleaned_lightweight = lightweight

#null values
average_reach_lightweight = cleaned_lightweight[cleaned_lightweight['division'] == 'Lightweight']['reach'].mean()
cleaned_lightweight['reach'].fillna(average_reach_lightweight, inplace=True)

average_age_lightweight = cleaned_lightweight[cleaned_lightweight['division'] == 'Lightweight']['age'].mean()
cleaned_lightweight['age'].fillna(average_age_lightweight, inplace=True)

average_height_lightweight = cleaned_lightweight[cleaned_lightweight['division'] == 'Lightweight']['height'].mean()
cleaned_lightweight['height'].fillna(average_height_lightweight, inplace=True)

#replace
def time_to_seconds(time_str):
    if pd.isnull(time_str):
        return np.nan
    minutes, seconds = map(int, time_str.split(':'))
    return minutes * 60 + seconds

specific_value = 216.1080438291005
cleaned_lightweight.loc[cleaned_lightweight['days_since_last_comp'] == specific_value, 'days_since_last_comp'] = cleaned_lightweight['age']


cleaned_lightweight['time_seconds'] = cleaned_lightweight['time'].apply(time_to_seconds)

#remove columns
columns_to_ignore = ['date', 'dob', 'division', 'referee', 'time', 'time_format']
cleaned_lightweight.drop(columns=columns_to_ignore, inplace=True)


/tmp/ipykernel_19/540385160.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_lightweight['reach'].fillna(average_reach_lightweight, inplace=True)
/tmp/ipykernel_19/540385160.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_lightweight['age'].fillna(average_age_lightweight, inplace=True)
/tmp/ipykernel_19/540385160.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_lightweight['height'].fillna(average_hei

In [9]:
#label encoding 
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Label encode 'stance' column
if 'stance' in cleaned_lightweight.columns:
    cleaned_lightweight['stance_encoded'] = label_encoder.fit_transform(cleaned_lightweight['stance'])

# Label encode 'method' column
if 'method' in cleaned_lightweight.columns:
    cleaned_lightweight['method_encoded'] = label_encoder.fit_transform(cleaned_lightweight['method'])

# Drop the original 'stance' and 'method' columns
cleaned_lightweight.drop(columns=['stance', 'method'], inplace=True)


/tmp/ipykernel_19/2437616765.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_lightweight['stance_encoded'] = label_encoder.fit_transform(cleaned_lightweight['stance'])
/tmp/ipykernel_19/2437616765.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_lightweight['method_encoded'] = label_encoder.fit_transform(cleaned_lightweight['method'])
/tmp/ipykernel_19/2437616765.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the 

In [10]:
lightweight = cleaned_lightweight

In [11]:
lightweight

,result,fighter,opponent,round,reach,age,height,knockdowns,days_since_last_comp,sub_attempts,...,lose_streak,win_loss_ratio,KO_losses,total_comp_time,stamina,elo,trueskill,time_seconds,stance_encoded,method_encoded
198,0,rainy martinez,jerry bohlander,1,71.469443,10727.379296,72.0,0,10727.379296,0,...,1,0.000000,0,84.0,0.000000,1155.936792,21.246039,84,0,3
199,1,jerry bohlander,rainy martinez,1,71.469443,8396.000000,71.0,0,140.000000,1,...,0,0.750000,1,1858.0,0.000000,1292.341803,29.965180,84,0,3
200,0,wallid ismail,yoshiki takahashi,2,71.469443,10577.000000,67.0,0,10577.000000,0,...,1,0.000000,0,900.0,0.000000,1147.756618,20.794729,180,0,4
201,1,yoshiki takahashi,wallid ismail,2,71.469443,10193.000000,71.0,0,10193.000000,0,...,0,1.000000,0,900.0,0.000000,1252.243382,29.205271,180,1,4
206,0,nick sanzo,jerry bohlander,1,71.469443,10727.379296,69.0,0,10727.379296,0,...,1,0.000000,0,39.0,0.000000,1162.881928,21.710767,39,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14055,1,justin gaethje,rafael fiziev,3,70.000000,12542.000000,71.0,0,315.000000,0,...,0,0.636364,2,6670.0,0.600000,1491.524082,38.809578,300,0,1
14108,0,lando vannata,daniel zellhuber,3,71.000000,11354.000000,69.0,0,357.000000,0,...,2,0.307692,0,9089.0,0.222222,1148.578353,26.265970,300,0,4
14109,1,daniel zellhuber,lando vannata,3,77.000000,8683.000000,73.0,1,210.000000,0,...,0,0.500000,0,1800.0,0.500000,1200.212327,26.476804,300,2,4
14118,0,clay guida,rafa garcia,3,70.000000,15103.000000,67.0,0,133.000000,0,...,1,0.514286,2,24276.0,0.521739,1178.957220,31.316986,300,0,4


# LinUCB 

### copy and edit/optimise after this markdown

In [12]:
lightweight.dtypes

result                         int64
fighter                       object
opponent                      object
round                          int64
reach                        float64
age                          float64
height                       float64
knockdowns                     int64
days_since_last_comp         float64
sub_attempts                   int64
reversals                      int64
control                        int64
takedowns_landed               int64
takedowns_attempts             int64
sig_strikes_landed             int64
sig_strikes_attempts           int64
total_strikes_landed           int64
total_strikes_attempts         int64
head_strikes_landed            int64
head_strikes_attempts          int64
body_strikes_landed            int64
body_strikes_attempts          int64
leg_strikes_landed             int64
leg_strikes_attempts           int64
distance_strikes_landed        int64
distance_strikes_attempts      int64
clinch_strikes_landed          int64
c

The new code

In [13]:
import numpy as np
import pandas as pd

class ModifiedWinLossLinUCB:
    def __init__(self, num_arms, num_features, alpha=0.05):
        self.num_arms = num_arms  # This is actually the number of fights, not fighters
        self.num_features = num_features
        self.alpha = alpha
        self.A = [np.identity(num_features) for _ in range(self.num_arms)]  # One A matrix per fight
        self.b = [np.zeros((num_features, 1)) for _ in range(self.num_arms)]  # One b vector per fight

    def recommend(self, context):
        chosen_arm = None
        max_ucb = float('-inf')

        for arm in [0, 1]:  # Two possible arms: fighter 1 wins or fighter 2 wins
            x = context.reshape((self.num_features, 1))
            A_inv = np.linalg.inv(self.A[arm])
            theta = np.dot(A_inv, self.b[arm])
            ucb_value = np.dot(x.T, np.dot(A_inv, x))

            # Calculate the UCB
            # Use item() to extract a scalar from a numpy array
            ucb = np.dot(theta.T, x) + self.alpha * np.sqrt(ucb_value.item())

            if ucb > max_ucb:
                max_ucb = ucb
                chosen_arm = arm

        return chosen_arm


    def update(self, fight_index, context, reward):
        x = context.reshape((self.num_features, 1))
        self.A[fight_index] += np.dot(x, x.T)
        self.b[fight_index] += reward * x


# Assume 'lightweight' is a pandas DataFrame containing your data
# Assume 'chosen_features' is defined as the list of features for the fighters
chosen_features = [
    "total_comp_time", "reach", "height", "age",
    "knockdowns", "sub_attempts", "reversals", "control",
    "takedowns_landed", "takedowns_attempts",
    "sig_strikes_landed", "sig_strikes_attempts",
    "total_strikes_landed", "total_strikes_attempts",
    "head_strikes_landed", "head_strikes_attempts",
    "body_strikes_landed", "body_strikes_attempts",
    "leg_strikes_landed", "leg_strikes_attempts",
    "distance_strikes_landed", "distance_strikes_attempts",
    "clinch_strikes_landed", "clinch_strikes_attempts",
    "ground_strikes_landed", "ground_strikes_attempts",
    "KO_losses", "days_since_last_comp", "lose_streak",
    "win_streak", "win_loss_ratio",
    "stamina", "num_fights", "trueskill", "elo"
]

# Ensure 'result' column exists and indicates the winner (1 for the winner, 0 for the loser)

num_fights = len(lightweight) // 2
num_features = len(chosen_features) * 2  # Features from both fighters

# Initialize variables to track accuracy in chunks
chunk_size = 100  # Define the size of each chunk for reporting accuracy
chunk_rewards = [0] * (num_fights // chunk_size + 1)  # List to hold rewards for each chunk
chunk_correct_predictions = [0] * (num_fights // chunk_size + 1)  # List to hold correct predictions count for each chunk

bandit = ModifiedWinLossLinUCB(num_fights, num_features)

total_reward = 0
correct_predictions = 0

print("Timestep | Chosen Arm | Actual Winner | Reward")
for fight_index in range(num_fights):
    # Get the records for both fighters in the current fight
    fighter1_record = lightweight.iloc[fight_index * 2]
    fighter2_record = lightweight.iloc[fight_index * 2 + 1]

    # Convert features from both fighters to numeric types and handle non-numeric entries
    fighter1_features = pd.to_numeric(fighter1_record[chosen_features], errors='coerce').fillna(0)
    fighter2_features = pd.to_numeric(fighter2_record[chosen_features], errors='coerce').fillna(0)

    context = np.concatenate([fighter1_features, fighter2_features])
    # Predict the winner using the combined context
    chosen_arm = bandit.recommend(context)  # This will be 0 or 1

    # Determine the actual winner (arm 0 represents fighter 1 and arm 1 represents fighter 2)
    actual_winner = 0 if fighter1_record['result'] == 1 else 1
    
    print(fight_index)
    
    print("Fighter " + str(fighter1_record['fighter'] + " " + str(fighter1_record['result'])) + ", " + "Fighter " + str(fighter2_record['fighter'] + " " + str(fighter2_record['result'])))
    
    winning_fighter = "name"
    if actual_winner == 0:
        winning_fighter = fighter1_record['fighter']
    else:
        winning_fighter = fighter2_record['fighter']
        
    print(f"Actual Winner: Fighter " + winning_fighter)


    # Update the bandit with the outcome of the fight
    reward = 1 if chosen_arm == actual_winner else 0
    bandit.update(fight_index // 2, context, reward)  # Update using fight index
    
    # Determine the current chunk
    current_chunk = fight_index // chunk_size
    
    # Update rewards and correct predictions for the current chunk
    chunk_rewards[current_chunk] += reward
    if chosen_arm == actual_winner:
        chunk_correct_predictions[current_chunk] += 1
    
    # Print the timestep, rewards, and chosen arm
    print(f"{fight_index+1:8} | {chosen_arm:11} | {actual_winner:14} | {reward:6}")
    
    # Track total reward and correct predictions
    total_reward += reward
    correct_predictions += (chosen_arm == actual_winner)

# Print accuracy for each chunk
for i in range(len(chunk_rewards)):
    if (i + 1) * chunk_size <= num_fights:
        print(f"Accuracy for steps {i * chunk_size + 1} - {(i + 1) * chunk_size}: {chunk_correct_predictions[i] / chunk_size}")
    else:  # Handle the last chunk which might be smaller than chunk_size
        print(f"Accuracy for steps {i * chunk_size + 1} - {num_fights}: {chunk_correct_predictions[i] / (num_fights - i * chunk_size)}")
        
# Calculate accuracy
accuracy = correct_predictions / num_fights
print(f"Total reward: {total_reward}")
print(f"Accuracy: {accuracy}")


Timestep | Chosen Arm | Actual Winner | Reward
0
Fighter rainy martinez 0, Fighter jerry bohlander 1
Actual Winner: Fighter jerry bohlander
       1 |           0 |              1 |      0
1
Fighter wallid ismail 0, Fighter yoshiki takahashi 1
Actual Winner: Fighter yoshiki takahashi
       2 |           1 |              1 |      1
2
Fighter nick sanzo 0, Fighter jerry bohlander 1
Actual Winner: Fighter jerry bohlander
       3 |           1 |              1 |      1
3
Fighter wes albritton 0, Fighter tito ortiz 1
Actual Winner: Fighter tito ortiz
       4 |           1 |              1 |      1
4
Fighter saeed hosseini 0, Fighter jack nilson 1
Actual Winner: Fighter jack nilson
       5 |           1 |              1 |      1
5
Fighter christophe leninger 0, Fighter guy mezger 1
Actual Winner: Fighter guy mezger
       6 |           1 |              1 |      1
6
Fighter royce alger 0, Fighter enson inoue 1
Actual Winner: Fighter enson inoue
       7 |           1 |              1 |   

In [14]:
import numpy as np
import pandas as pd

class ModifiedWinLossLinUCB:
    def __init__(self, num_arms, num_features, alpha=0.1):
        self.num_arms = num_arms  # This is actually the number of fights, not fighters
        self.num_features = num_features
        self.alpha = alpha
        self.A = [np.identity(num_features) for _ in range(self.num_arms)]  # One A matrix per fight
        self.b = [np.zeros((num_features, 1)) for _ in range(self.num_arms)]  # One b vector per fight

    def recommend(self, context):
        chosen_arm = None
        max_ucb = float('-inf')

        for arm in [0, 1]:  # Two possible arms: fighter 1 wins or fighter 2 wins
            x = context.reshape((self.num_features, 1))
            A_inv = np.linalg.inv(self.A[arm])
            theta = np.dot(A_inv, self.b[arm])
            ucb_value = np.dot(x.T, np.dot(A_inv, x))

            # Calculate the UCB
            # Use item() to extract a scalar from a numpy array
            ucb = np.dot(theta.T, x) + self.alpha * np.sqrt(ucb_value.item())

            if ucb > max_ucb:
                max_ucb = ucb
                chosen_arm = arm

        return chosen_arm


    def update(self, fight_index, context, reward):
        x = context.reshape((self.num_features, 1))
        self.A[fight_index] += np.dot(x, x.T)
        self.b[fight_index] += reward * x


# Assume 'lightweight' is a pandas DataFrame containing your data
# Assume 'chosen_features' is defined as the list of features for the fighters
chosen_features = [
    "total_comp_time", "reach", "height", "age",
    "knockdowns", "sub_attempts", "reversals", "control",
    "takedowns_landed", "takedowns_attempts",
    "sig_strikes_landed", "sig_strikes_attempts",
    "total_strikes_landed", "total_strikes_attempts",
    "head_strikes_landed", "head_strikes_attempts",
    "body_strikes_landed", "body_strikes_attempts",
    "leg_strikes_landed", "leg_strikes_attempts",
    "distance_strikes_landed", "distance_strikes_attempts",
    "clinch_strikes_landed", "clinch_strikes_attempts",
    "ground_strikes_landed", "ground_strikes_attempts",
    "KO_losses", "days_since_last_comp", "lose_streak",
    "win_streak", "win_loss_ratio",
    "stamina", "num_fights", "trueskill", "elo"
]

# Ensure 'result' column exists and indicates the winner (1 for the winner, 0 for the loser)

num_fights = len(welterweight) // 2
num_features = len(chosen_features) * 2  # Features from both fighters

# Initialize variables to track accuracy in chunks
chunk_size = 100  # Define the size of each chunk for reporting accuracy
chunk_rewards = [0] * (num_fights // chunk_size + 1)  # List to hold rewards for each chunk
chunk_correct_predictions = [0] * (num_fights // chunk_size + 1)  # List to hold correct predictions count for each chunk

bandit = ModifiedWinLossLinUCB(num_fights, num_features)

total_reward = 0
correct_predictions = 0

print("Timestep | Chosen Arm | Actual Winner | Reward")
for fight_index in range(num_fights):
    # Get the records for both fighters in the current fight
    fighter1_record = welterweight.iloc[fight_index * 2]
    fighter2_record = welterweight.iloc[fight_index * 2 + 1]

    # Convert features from both fighters to numeric types and handle non-numeric entries
    fighter1_features = pd.to_numeric(fighter1_record[chosen_features], errors='coerce').fillna(0)
    fighter2_features = pd.to_numeric(fighter2_record[chosen_features], errors='coerce').fillna(0)

    context = np.concatenate([fighter1_features, fighter2_features])
    # Predict the winner using the combined context
    chosen_arm = bandit.recommend(context)  # This will be 0 or 1

    # Determine the actual winner (arm 0 represents fighter 1 and arm 1 represents fighter 2)
    actual_winner = 0 if fighter1_record['result'] == 1 else 1
    
    print(fight_index)
    
    print("Fighter " + str(fighter1_record['fighter'] + " " + str(fighter1_record['result'])) + ", " + "Fighter " + str(fighter2_record['fighter'] + " " + str(fighter2_record['result'])))
    
    winning_fighter = "name"
    if actual_winner == 0:
        winning_fighter = fighter1_record['fighter']
    else:
        winning_fighter = fighter2_record['fighter']
        
    print(f"Actual Winner: Fighter " + winning_fighter)


    # Update the bandit with the outcome of the fight
    reward = 1 if chosen_arm == actual_winner else 0
    bandit.update(fight_index // 2, context, reward)  # Update using fight index
    
    # Determine the current chunk
    current_chunk = fight_index // chunk_size
    
    # Update rewards and correct predictions for the current chunk
    chunk_rewards[current_chunk] += reward
    if chosen_arm == actual_winner:
        chunk_correct_predictions[current_chunk] += 1
    
    # Print the timestep, rewards, and chosen arm
    print(f"{fight_index+1:8} | {chosen_arm:11} | {actual_winner:14} | {reward:6}")
    
    # Track total reward and correct predictions
    total_reward += reward
    correct_predictions += (chosen_arm == actual_winner)

# Print accuracy for each chunk
for i in range(len(chunk_rewards)):
    if (i + 1) * chunk_size <= num_fights:
        print(f"Accuracy for steps {i * chunk_size + 1} - {(i + 1) * chunk_size}: {chunk_correct_predictions[i] / chunk_size}")
    else:  # Handle the last chunk which might be smaller than chunk_size
        print(f"Accuracy for steps {i * chunk_size + 1} - {num_fights}: {chunk_correct_predictions[i] / (num_fights - i * chunk_size)}")
        
# Calculate accuracy
accuracy = correct_predictions / num_fights
print(f"Total reward: {total_reward}")
print(f"Accuracy: {accuracy}")


Timestep | Chosen Arm | Actual Winner | Reward
0
Fighter adrian serrano 0, Fighter shonie carter 1
Actual Winner: Fighter shonie carter
       1 |           0 |              1 |      0
1
Fighter marcelo aguiar 0, Fighter matt hughes 1
Actual Winner: Fighter matt hughes
       2 |           1 |              1 |      1
2
Fighter john alessio 0, Fighter pat miletich 1
Actual Winner: Fighter pat miletich
       3 |           1 |              1 |      1
3
Fighter laverne clark 0, Fighter fabiano iha 1
Actual Winner: Fighter fabiano iha
       4 |           1 |              1 |      1
4
Fighter chris lytle 0, Fighter ben earwood 1
Actual Winner: Fighter ben earwood
       5 |           1 |              1 |      1
5
Fighter matt hughes 0, Fighter dennis hallman 1
Actual Winner: Fighter dennis hallman
       6 |           0 |              1 |      0
6
Fighter daiju takase 0, Fighter fabiano iha 1
Actual Winner: Fighter fabiano iha
       7 |           0 |              1 |      0
7
Fighter keni